In [1]:
import pandas as pd 
import psycopg2
from investor_watch.constants import TICKER_PATH, STOCK_LIST_PATH
from investor_watch import db

conn = psycopg2.connect(                   # Connect to an existing database
    dbname="postgres", 
    user="postgres", 
    password="password", 
    host = 'localhost', 
    port = '5432'
)
cur = conn.cursor()                        # Open a cursor to perform database operations

table_schema = '''
CREATE TABLE IF NOT EXISTS stocks (
    ticker VARCHAR(10) PRIMARY KEY,
    company VARCHAR(255) NOT NULL,
    sector VARCHAR(100),
    industry VARCHAR(100),
    country VARCHAR(100),
    price NUMERIC(10, 2) CHECK (price >= 0),
    market_cap NUMERIC(10, 2) CHECK (market_cap >= 0)
);

CREATE TABLE IF NOT EXISTS articles (
    id SERIAL PRIMARY KEY,
    ticker VARCHAR(10) REFERENCES stocks(ticker) ON DELETE CASCADE,
    date TIMESTAMP NOT NULL,
    title VARCHAR(255) NOT NULL,
    link TEXT NOT NULL,
    source VARCHAR(100) NOT NULL,
    UNIQUE (ticker, date, title)                                    
);
'''

cur.execute(table_schema)



Tables initialized successfully
c:\users\patrick4\documents\projects\investor_watch\investor_watch\constants.py


Insert csv data into the sql tables

In [2]:
# stock_list.csv -> sql 
df = pd.read_csv(STOCK_LIST_PATH)

for _, row in df.iterrows():
    try:
        cur.execute("""
            INSERT INTO stocks (ticker, company, sector, industry, country, price, market_cap)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (ticker) DO NOTHING;
        """, (
            row['Ticker'],
            row['Company'],
            row['Sector'],
            row['Industry'],
            row['Country'],
            float(row['Price']),
            float(row['Market Cap (billions)'])
        ))
    except Exception as e:
        print(f"Error inserting {row['Ticker']}: {e}")

conn.commit() 


In [2]:
# ticker.csv -> sql
import os 
from investor_watch.constants import TICKER_PATH

# Get all ticker files from the tickers directory
ticker_files = [f for f in os.listdir(TICKER_PATH) if f.endswith('.csv')]

# Process each ticker file
for ticker_file in ticker_files:
    ticker = ticker_file.split('.')[0]  # Extract ticker from filename
    file_path = os.path.join(TICKER_PATH, ticker_file)
    
    try:
        df = pd.read_csv(file_path)
        print(f"Processing {ticker} with {len(df)} articles...")
        
        for _, row in df.iterrows():
            cur.execute("""
                INSERT INTO articles (ticker, date, title, link, source)
                VALUES (%s, %s, %s, %s, %s)
                ON CONFLICT (ticker, date, title) DO NOTHING;
            """, (
                ticker,
                pd.to_datetime(row['Date']).to_pydatetime(),
                row['Title'],
                row['Link'],
                row['Source']
            ))
        conn.commit()         
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        conn.rollback()
        




Processing A with 4 articles...
Processing AA with 5 articles...
Error processing AA: insert or update on table "articles" violates foreign key constraint "articles_ticker_fkey"
DETAIL:  Key (ticker)=(AA) is not present in table "stocks".

Processing AAON with 0 articles...
Processing AAPL with 170 articles...
Processing ABBV with 5 articles...
Processing ABNB with 13 articles...
Processing ABT with 3 articles...
Processing ACI with 8 articles...
Processing ACM with 0 articles...
Processing ADBE with 24 articles...
Processing ADI with 4 articles...
Processing ADM with 27 articles...
Processing ADP with 2 articles...
Processing ADSK with 11 articles...
Processing AEE with 1 articles...
Processing AEP with 3 articles...
Processing AFG with 0 articles...
Processing AFL with 1 articles...
Processing AFRM with 8 articles...
Processing AGR with 1 articles...
Error processing AGR: insert or update on table "articles" violates foreign key constraint "articles_ticker_fkey"
DETAIL:  Key (ticker)